In [1]:
!pip install tensorflow opencv-python firebase-admin numpy

                                              0.0/120.3 kB ? eta -:--:--
     ---                                      10.2/120.3 kB ? eta -:--:--
     ---------                             30.7/120.3 kB 435.7 kB/s eta 0:00:01
     ---------                             30.7/120.3 kB 435.7 kB/s eta 0:00:01
     ------------                          41.0/120.3 kB 196.9 kB/s eta 0:00:01
     -------------------------             81.9/120.3 kB 353.1 kB/s eta 0:00:01
     ---------------------------------    112.6/120.3 kB 435.7 kB/s eta 0:00:01
     ------------------------------------ 120.3/120.3 kB 413.8 kB/s eta 0:00:00
                                              0.0/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
                                              0.1/12.0 MB ? eta -:--:--
             

# Register New User

In [2]:
import cv2
import firebase_admin
from firebase_admin import credentials, firestore, storage, initialize_app
from keras_facenet import FaceNet

# Initialize Firebase Admin
cred_path = 'C:/Users/hp/Desktop/Crater Co/Attendance App/attendance-facerecog-firebase-adminsdk-pqhqh-a047efc55c.json'

# Check if app already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(cred_path)
    default_app = initialize_app(cred, {
        'storageBucket': 'gs://attendance-facerecog.appspot.com'
    })
else:
    default_app = firebase_admin.get_app()

db = firestore.client()
bucket = storage.bucket('attendance-facerecog.appspot.com')

# Initialize the FaceNet embedder
embedder = FaceNet()

# Function to upload image to Firebase Storage
def upload_image(image_path):
    blob = bucket.blob(image_path)
    blob.upload_from_filename(image_path)
    # Make the blob publicly accessible
    blob.make_public()
    return blob.public_url

# Function to get embeddings using keras-facenet
def get_embeddings(image_path):
    detections = embedder.extract(image_path, threshold=0.95)
    if detections:
        # Assuming the first face detected is the user
        return detections[0]['embedding']
    else:
        return None

## Run this for registering

In [3]:
# Prompt for Employee Name
employee_name = input("Enter Name: ")

# Capture Image from Webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

if ret:
    # Save image locally
    img_name = f"{employee_name.replace(' ', '_')}.jpg"
    cv2.imwrite(img_name, frame)

    # Get embeddings for the captured image
    embeddings = get_embeddings(frame)
    if embeddings is not None:
        # Convert embedding numpy array to list for Firestore
        embedding_list = embeddings.tolist()
        
        # Upload to Firebase Storage
        image_url = upload_image(img_name)
        
        # Add to Firestore including embedding
        doc_ref = db.collection('employees').document(employee_name)
        doc_ref.set({
            'name': employee_name,
            'image_url': image_url,
            'embedding': embedding_list  # Store the embedding
        })
        print(f"Uploaded {employee_name}'s info successfully.")
    else:
        print("No face detected or embedding generation failed.")
else:
    print("Failed to capture image")


Enter Name: Azmain
1/1 [==============================] - 4s 4s/step
Uploaded Azmain's info successfully.


## Registration GUI

In [20]:
import tkinter as tk
from tkinter import messagebox
import cv2
from PIL import Image, ImageTk
import threading

# Function to handle the capture and upload process
def handle_capture():
    employee_name_input = employee_name.get()
    if not employee_name_input:
        messagebox.showerror("Error", "Please enter the employee's name.")
        return
    
    # Start the capture process in a separate thread to keep the GUI responsive
    threading.Thread(target=capture_and_upload, args=(employee_name_input,)).start()

def capture_and_upload(employee_name_input):
    # Capture Image from Webcam
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()

    if ret:
        # Save image locally
        img_name = f"{employee_name_input.replace(' ', '_')}.jpg"
        cv2.imwrite(img_name, frame)

        # Get embeddings for the captured image
        embeddings = get_embeddings(img_name)  # Use the saved image path
        if embeddings is not None:
            # Convert embedding numpy array to list for Firestore
            embedding_list = embeddings.tolist()

            # Upload to Firebase Storage
            image_url = upload_image(img_name)

            # Add to Firestore including embedding
            doc_ref = db.collection('employees').document(employee_name_input)
            doc_ref.set({
                'name': employee_name_input,
                'image_url': image_url,
                'embedding': embedding_list  # Store the embedding
            })
            print(f"Uploaded {employee_name_input}'s info successfully.")
            messagebox.showinfo("Success", f"Uploaded {employee_name_input}'s info successfully.")
        else:
            messagebox.showerror("Error", "No face detected or embedding generation failed.")
    else:
        messagebox.showerror("Error", "Failed to capture image.")

# GUI setup
window = tk.Tk()
window.title("Employee Registration")

# Employee name input
tk.Label(window, text="Enter Employee Name:").pack(padx=10, pady=5)
employee_name = tk.Entry(window)
employee_name.pack(padx=10, pady=5)

# Capture button
capture_btn = tk.Button(window, text="Register", command=handle_capture)
capture_btn.pack(padx=10, pady=10)

window.mainloop()


1/1 [==============================] - 0s 120ms/step
Uploaded Az's info successfully.


# Take Attendance

In [10]:
import cv2
import numpy as np
from datetime import datetime
from scipy.spatial import distance
from firebase_admin import firestore
from keras_facenet import FaceNet

# Initialize Firebase Admin
cred_path = 'C:/Users/hp/Desktop/Crater Co/Attendance App/attendance-facerecog-firebase-adminsdk-pqhqh-a047efc55c.json'

# Check if app already initialized
if not firebase_admin._apps:
    cred = credentials.Certificate(cred_path)
    default_app = initialize_app(cred, {
        'storageBucket': 'gs://attendance-facerecog.appspot.com'
    })
else:
    default_app = firebase_admin.get_app()

db = firestore.client()
bucket = storage.bucket('attendance-facerecog.appspot.com')

# Initialize the FaceNet embedder
embedder = FaceNet()

# Function to calculate Euclidean distance for embeddings
def find_closest_match(embedding, registered_embeddings):
    min_dist = float('inf')
    matched_user_id = None
    for user_id, registered_embedding in registered_embeddings.items():
        dist = distance.euclidean(embedding, registered_embedding)
        if dist < min_dist:
            min_dist = dist
            matched_user_id = user_id
    return matched_user_id

# Function to fetch registered users' embeddings from Firestore
def fetch_registered_embeddings():
    users_ref = db.collection('employees').stream()
    registered_embeddings = {}
    for user in users_ref:
        user_info = user.to_dict()
        registered_embeddings[user.id] = user_info['embedding']
    return registered_embeddings

# Function to store attendance record
def store_attendance_record(user_id, image_url):
    timestamp = datetime.now()
    attendance_ref = db.collection('Attendance').document()
    attendance_ref.set({
        'user_id': user_id,
        'image_url': image_url,
        'timestamp': timestamp
    })
    print(f"Attendance recorded for {user_id} at {timestamp}.")

## Run this to take attendance

In [17]:
# Capture Image from Webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

if ret:
    # Generate embedding for the captured image
    captured_embedding = get_embeddings(frame)
    
    if captured_embedding is not None:
        # Fetch registered embeddings from Firestore
        registered_embeddings = fetch_registered_embeddings()

        # Find the closest match for the captured embedding
        matched_user_id = find_closest_match(captured_embedding, registered_embeddings)

        if matched_user_id:
            # Generate a timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            # Save image locally with the employee name and timestamp
            img_name = f"{matched_user_id}_{timestamp}.jpg"
            cv2.imwrite(img_name, frame)

            # Upload to Firebase Storage
            image_url = upload_image(img_name)

            # Store the attendance record with the image URL and timestamp
            store_attendance_record(matched_user_id, image_url)
        else:
            print("No registered user matched.")
    else:
        print("No face detected or embedding generation failed.")
else:
    print("Failed to capture image")

1/1 [==============================] - 0s 33ms/step
No face detected or embedding generation failed.


## Attendance GUI

In [23]:
import tkinter as tk
from tkinter import messagebox
import threading
import cv2

def capture_and_record_attendance():
    def run():
        update_status("Capturing image...")
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        cap.release()

        if ret:
            update_status("Processing image...")
            captured_embedding = get_embeddings(frame)

            if captured_embedding is not None:
                registered_embeddings = fetch_registered_embeddings()
                matched_user_id = find_closest_match(captured_embedding, registered_embeddings)

                if matched_user_id:
                    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                    img_name = f"{matched_user_id}_{timestamp}.jpg"
                    cv2.imwrite(img_name, frame)
                    image_url = upload_image(img_name)
                    store_attendance_record(matched_user_id, image_url)
                    update_status(f"Attendance recorded for {matched_user_id}.")
                else:
                    update_status("No registered user matched.")
            else:
                update_status("No face detected or embedding generation failed.")
        else:
            update_status("Failed to capture image.")

    # Run the capture and record process in a separate thread
    threading.Thread(target=run).start()

def update_status(message):
    # This function updates the status label with the provided message
    status_label.config(text=message)
    # Force the GUI to update the status label immediately
    window.update_idletasks()

# Create the main window (GUI)
window = tk.Tk()
window.title("Attendance System")

# Create a button to start the attendance recording process
capture_button = tk.Button(window, text="Record Attendance", command=capture_and_record_attendance)
capture_button.pack(pady=20)

# Create a label to display the status
status_label = tk.Label(window, text="Ready")
status_label.pack(pady=10)

# Start the GUI event loop
window.mainloop()


1/1 [==============================] - 0s 97ms/step
Attendance recorded for Az at 2024-02-13 20:15:53.403551.


In [1]:
!pip install Flask flask-cors

In [1]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import firebase_admin
from firebase_admin import credentials, firestore, storage, initialize_app
from keras_facenet import FaceNet
import numpy as np
from datetime import datetime
from scipy.spatial import distance
import cv2
import os
from werkzeug.utils import secure_filename

# Initialize Flask App
app = Flask(__name__)
CORS(app)

# Initialize Firebase Admin
cred_path = 'C:/Users/hp/Desktop/Crater Co/Attendance App/attendance-facerecog-firebase-adminsdk-pqhqh-a047efc55c.json'
cred = credentials.Certificate(cred_path)
firebase_admin.initialize_app(cred, {
    'storageBucket': 'attendance-facerecog.appspot.com'
})

db = firestore.client()
bucket = storage.bucket()

# Initialize the FaceNet embedder
embedder = FaceNet()

# Function to upload image to Firebase Storage
def upload_image(image_path):
    blob = bucket.blob(image_path)
    blob.upload_from_filename(image_path)
    blob.make_public()
    return blob.public_url

# Function to get embeddings using keras-facenet
def get_embeddings(image):
    detections = embedder.extract(image, threshold=0.95)
    if detections:
        return detections[0]['embedding']
    else:
        return None

# Function to fetch registered users' embeddings from Firestore
def fetch_registered_embeddings():
    users_ref = db.collection('employees').stream()
    registered_embeddings = {}
    for user in users_ref:
        user_info = user.to_dict()
        registered_embeddings[user.id] = user_info['embedding']
    return registered_embeddings

# Function to calculate Euclidean distance for embeddings
def find_closest_match(embedding, registered_embeddings):
    min_dist = float('inf')
    matched_user_id = None
    for user_id, registered_embedding in registered_embeddings.items():
        dist = distance.euclidean(embedding, registered_embedding)
        if dist < min_dist:
            min_dist = dist
            matched_user_id = user_id
    return matched_user_id

# Function to store attendance record
def store_attendance_record(user_id, image_url):
    timestamp = datetime.now()
    attendance_ref = db.collection('Attendance').document()
    attendance_ref.set({
        'user_id': user_id,
        'image_url': image_url,
        'timestamp': timestamp
    })

@app.route('/register', methods=['POST'])
def register_student():
    file = request.files['file']
    name = request.form['name']
    if file and name:
        filename = secure_filename(name + '.jpg')
        file_path = os.path.join('C:/Users/hp/Desktop/Crater Co/Attendance App/DemoFlaskImg', filename)
        file.save(file_path)

        embeddings = get_embeddings(file_path)
        if embeddings is not None:
            embedding_list = embeddings.tolist()
            image_url = upload_image(file_path)
            
            # Add to Firestore including embedding
            doc_ref = db.collection('employees').document(name)
            doc_ref.set({
                'name': name,
                'image_url': image_url,
                'embedding': embedding_list
            })
            return jsonify({'message': f"Uploaded {name}'s info successfully."}), 200
        else:
            return jsonify({'message': "No face detected or embedding generation failed."}), 400
    else:
        return jsonify({'message': "Missing name or file."}), 400

@app.route('/record_attendance', methods=['POST'])
def record_attendance():
    file = request.files['file']
    if file:
        filename = secure_filename('temp.jpg')
        file_path = os.path.join('C:/Users/hp/Desktop/Crater Co/Attendance App/DemoFlaskImg', filename)
        file.save(file_path)

        captured_embedding = get_embeddings(file_path)
        if captured_embedding is not None:
            registered_embeddings = fetch_registered_embeddings()
            matched_user_id = find_closest_match(captured_embedding, registered_embeddings)
            if matched_user_id:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                img_name = f"{matched_user_id}_{timestamp}.jpg"
                image_url = upload_image(img_name)

                store_attendance_record(matched_user_id, image_url)
                return jsonify({'message': f"Attendance recorded for {matched_user_id} at {timestamp}."}), 200
            else:
                return jsonify({'message': "No registered user matched."}), 400
        else:
            return jsonify({'message': "No face detected or embedding generation failed."}), 400
    else:
        return jsonify({'message': "Missing file."}), 400

if __name__ == '__main__':
    app.run()





 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


1/1 [==============================] - 2s 2s/step


127.0.0.1 - - [18/Feb/2024 20:29:18] "POST /register HTTP/1.1" 200 -
